In [1]:
import os

ACTUAL_PATH = os.getcwd()
NEW_PATH = os.path.dirname(ACTUAL_PATH)

os.chdir(NEW_PATH)

print(ACTUAL_PATH)
print(NEW_PATH)

c:\Users\lejcr\projetos\cc_fraud_detection_with_faker\cc_fraud_detection_with_faker\src\notebooks
c:\Users\lejcr\projetos\cc_fraud_detection_with_faker\cc_fraud_detection_with_faker\src


In [2]:
import math
import os
import random
from datetime import date, datetime, timedelta

import faker
import pandas as pd
from faker import Faker

from sparkov_data_generation.datagen_customer import generate_customers
from sparkov_data_generation.datagen_transaction import generate_transactions

In [3]:
profiles = [
    p
    for p in os.listdir("sparkov_data_generation/profiles")
    if p.startswith("adults") or (p.startswith("young"))
]
profiles

['adults_2550_female_rural.json',
 'adults_2550_female_urban.json',
 'adults_2550_male_rural.json',
 'adults_2550_male_urban.json',
 'adults_50up_female_rural.json',
 'adults_50up_female_urban.json',
 'adults_50up_male_rural.json',
 'adults_50up_male_urban.json',
 'young_adults_female_rural.json',
 'young_adults_female_urban.json',
 'young_adults_male_rural.json',
 'young_adults_male_urban.json']

In [4]:
# Faker seed number to ensure constant customer list
seed_num = 161809

# initial number of customers
initial_cust_base_size = 3000

# generate n months worth of data
n_months = 12
start_dt = datetime.strptime("2021-01-01", "%Y-%m-%d")
end_dt = start_dt + pd.offsets.MonthEnd(n_months)

cust_list = pd.DataFrame()
# txn_df = pd.DataFrame()

# output directory and file naming conventions
data_base_dir = "data/"
customer_output_file = f"{data_base_dir}/customers_list.csv"
txn_output_file = "_txn.csv"

# create output directory if not existing
os.makedirs(data_base_dir, exist_ok=True)

In [5]:
fake = Faker()
Faker.seed(seed_num)

In [6]:
for dt in (start_dt + timedelta(n) for n in range(int((end_dt - start_dt).days))):
    print("Date", dt)

    if dt == start_dt:
        num_new_customers = initial_cust_base_size
    else:
        increment = random.uniform(0.01, 0.03)
        num_new_customers = math.ceil(len(cust_list) * increment)

    print("Number of customers newly onboard: ", num_new_customers)

    new_cust_list = generate_customers(
        fake, num_new_customers, "sparkov_data_generation/profiles/main_config.json"
    )

    cust_list = pd.concat([cust_list, new_cust_list], ignore_index=True)

    print("Total customer: ", len(cust_list))

    new_cust_list.to_csv(
        customer_output_file,
        mode="a",
        header=not os.path.exists(customer_output_file),
        index=False,
    )

    for p in profiles:
        # generate txns for the new customers from current date to the end date
        new_cust_txn_df = generate_transactions(new_cust_list, p, dt, end_dt)

        print(f"Number of transaction ({p}): ", len(new_cust_txn_df))
        
        new_cust_txn_file = p.split('.json')[0] + txn_output_file
        
        new_cust_txn_df.to_csv(
            os.path.join(data_base_dir, new_cust_txn_file),
            mode="a",
            header=not os.path.exists(f"{data_base_dir}{p}{txn_output_file}"),
            index=False,
        )
        

Date 2021-01-01 00:00:00
Number of customers newly onboard:  3000
Total customer:  3000


In [ ]:
print(f"Number of customers: {len(cust_list)}")